In [ ]:
import numpy as np
from fdm import plot_mesh, plot_contour, Laplaciano2D, RHS

In [ ]:
#
# 0       1       2       3       4       5   
# |                                       |   Nx  = 4 (nodos interiores)
# *---*---|---*---|---*---|---*---|---*---*   Nvx = 5 (volúmenes)
# |                                       |
# 0   1       2       3       4       5   6   hx = L / Nx + 1 = L / Nvx 
#
#  <----------------- L ------------------>  
#
Nx = 10
Ny = 21
Lx = 1.0
Ly = 2.0
hx = Lx / (Nx+1)
hy = Ly / (Ny+1)
h = hx
k = 1.0
print(hx, hy)
print(Nx, Ny)

In [ ]:
# Nodos totales incluyendo las fronteras
NxT = Nx + 2
NyT = Ny + 2

print(NxT, NyT)

# Coordenadas de la malla
xn = np.linspace(0,Lx,NxT)
yn = np.linspace(0,Ly,NyT)
xg, yg = np.meshgrid(xn, yn, indexing='ij', sparse=False)
plot_mesh(Lx, Ly, xg, yg)

In [ ]:
# Definición del campo escalar
T = np.zeros((NxT, NyT))

# Valores en las fronteras del dominio
TL = 0.0; TR = 0.0
TB = 100.0; TT = 0.0

T[0 , :] = TL # LEFT
T[-1, :] = TR # RIGHT
T[: , 0] = TB # BOTTOM
T[: ,-1] = TT # TOP

plot_contour(Lx, Ly, xg, yg, T, ticks = [0, TL, TR, TB, TT], cmap='viridis', yshared=True)

In [ ]:
%%time
# Método de Euler hacia adelante
ht = 0.001
r = k * ht / h**2
T_new = T.copy()
tolerancia = 9.0e-2
error = 1.0
error_lista = []

count = 1
while(error > tolerancia):
    for i in range(1,Nx+1):
        for j in range(1,Ny+1):
            T_new[i,j] = T[i,j] + r * (T[i+1,j] + T[i-1,j] + T[i,j+1] + T[i,j-1] - 4*T[i,j])
    error = np.linalg.norm(T_new - T)
    print(count, error)
    error_lista.append(error)
    T[:] = T_new[:]
    count += 1
    
#print(count, error)

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(12,5))

plt.subplot(1,3,1)
plot_mesh(Lx, Ly, xg, yg)

plt.subplot(1,3,2)
plot_contour(Lx, Ly, xg, yg, T, ticks = [0, 50, 100], 
             mesh = True, lines = 10, colors = 'white',
             cmap='viridis', yshared=True)

plt.subplot(1,3,3)
plt.plot(error_lista)
plt.yscale('log')
plt.ylabel('Error')
plt.xlabel('Iteración')
plt.tight_layout()